In [1]:
import cv2 as cv
from matplotlib import pyplot as plt
import numpy as np
from scipy import stats
import math
import ipywidgets as widgets

In [4]:
# Get's the disparity of the stereo images given the stereo images 
# return: disparity map (ndarray of the same size of image)
def get_disparity(fileL, fileR, num_disp, block_size): 

  imgL = cv.imread(fileL, cv.IMREAD_GRAYSCALE)
  imgR = cv.imread(fileR, cv.IMREAD_GRAYSCALE)
  stereo = cv.StereoBM_create(numDisparities=num_disp, blockSize = block_size)
  disparity = stereo.compute(imgL, imgR) 

  return disparity, stereo

In [5]:
 # Naive fills the disparity map by finding the mode of a block of the map 
 # Sets the filled disparity map to be the mode of the block that's not -16 
def naive_fill(disparity, block_size):
  # traverse disparity 
  filled = np.zeros(disparity.shape)

  for r in range(disparity.shape[0]): 
    for c in range(disparity.shape[1]): 

      # Take most common value in the block 
      block = disparity[max(0, r - block_size) : min(disparity.shape[0] - 1, r + block_size), max(0, c - block_size) : min(disparity.shape[1] - 1, c + block_size)]
      block = np.reshape(block, block.shape[0] * block.shape[1])
      block = np.delete(block, np.where(block == -16))

      # Set most common to be the value most commonly appearing in the block that isn't -16
      most_common = 0
      if len(block) != 0: 
        most_common = stats.mode(block)[0]

      # set value in filled to be most common 
      filled[r][c] = most_common 

  return filled

In [6]:
def regularize(fileL, fileR, left_matcher=None, left_disparity=None, lamb=8000, sigmaColor=1.4):
  cv_imgL = cv.imread(fileL, cv.IMREAD_GRAYSCALE)
  cv_imgR = cv.imread(fileR, cv.IMREAD_GRAYSCALE)
  
  
  if left_matcher is None:
    left_matcher = cv.StereoBM_create(numDisparities=16*3, blockSize=5)
  
  wls_filter = cv.ximgproc.createDisparityWLSFilter(left_matcher)
  right_matcher = cv.ximgproc.createRightMatcher(left_matcher)

  if left_disparity is None:
    left_disparity = left_matcher.compute(cv_imgL, cv_imgR)
  right_disparity = right_matcher.compute(cv_imgR, cv_imgL)

  wls_filter.setLambda(lamb)
  wls_filter.setSigmaColor(sigmaColor)

  filtered = wls_filter.filter(left_disparity, cv_imgL, disparity_map_right=right_disparity)

  return filtered

In [7]:
# Normalize the disparity map so that all values are between 0 and 1
def normalize_disparity(disparity): 
  max = np.amax(disparity) 

  return (1 / max) * disparity

In [8]:
# Calculate how far something is relationally 
# Uses an exponential function 
def calc_sigma(depth, focus_point, exp_base):
  return exp_base ** (abs(depth - focus_point)) - 1

In [9]:
# Calculate the kernel size based on the sigma value
# n = ceiling(6*sigma) + (ceiling(6 * sigma) + 1) % 2
def calc_gaussian_kernel_size(sigma): 
  return math.ceil(6 * sigma) + (math.ceil(6 * sigma) + 1) % 2 

In [10]:
# Wraps the cv Gaussian Blur function
def blur_image(image, depth, focus_depth, exp_base): 
  sigma = calc_sigma(depth, focus_depth, exp_base) 
  kernel_size = calc_gaussian_kernel_size(sigma)
  return cv.GaussianBlur(image, ksize=(kernel_size,kernel_size), sigmaX=sigma, borderType=cv.BORDER_REPLICATE)

In [11]:
# programmatically handle the blur of different depths 
def blur_images(image, disparity_map, focus_point, num_partitions, exp_base):
  max = np.amax(disparity_map) 

  partitions = num_partitions

  focus = focus_point
  
  blurred_images = [] 
  blurred_depths = [] 

  for i in range(partitions): 
    depth = (i + 1) / partitions 

    blurred_depths.append(depth)
    blurred_images.append(blur_image(image, depth, focus, exp_base))

  return blurred_images, blurred_depths

In [15]:
def simulate_lens_blur(fileL, fileR, num_disp, block_size=5, focus_point=0.5, num_partitions=10, exp_base=70):
  disparity, stereo = get_disparity(fileL, fileR, num_disp, block_size)
  filled = regularize(fileL, fileR, stereo, disparity)
  # normalize the disparity map so that disparity values are between 0 and 1
  filled = normalize_disparity(filled)

  # load image in color 
  img = cv.imread(fileL, 1)

  # store different blurred images in array
  blurred_images, blurred_depths = blur_images(img, filled, focus_point, num_partitions, exp_base)

  # overlay different depth blurs 
  composite_image = np.zeros(img.shape)
  front_image = np.zeros(img.shape)

  # iterate over filled
  # depending on which depth value the pixel has, assign new pixel to composite image 
  for r in range(img.shape[0]): 
    for c in range(img.shape[1]): 
      target_depth = filled[r][c] 

      i = 0 
      for depth in blurred_depths: 
        if target_depth <= depth: 
          break
        i += 1

      composite_image[r][c] = blurred_images[i][r][c]
      if i == 0: 
        front_image[r][c] = filled[r][c]
  return np.array(composite_image[:,num_disp:,], np.uint8)

In [16]:
class stereoblurUI:
  def __init__(self):

    self.fl_in = widgets.Text(
        value='cones/im2.png',
        placeholder='cones/im2.png',
        description='Left Image:'
    )    
    self.fr_in = widgets.Text(
        value='cones/im6.png',
        placeholder='cones/im6.png',
        description='Right Image:'
    )
    self.fileui = widgets.VBox([self.fl_in, self.fr_in])

    
    self.dsp_s = widgets.IntSlider(
          value=16*3,
          min=16*3,
          max=16*10,
          step=16
    )
    self.blk_s = widgets.IntSlider(
          value=5,
          min=5,
          max=20,
          step=2
    )

    self.depthmapui = widgets.HBox([widgets.VBox([widgets.Label('Num of Disparities:'),
                                                  widgets.Label('        Block Size:')]),
                                    widgets.VBox([self.dsp_s, self.blk_s])])

    self.exp_s = widgets.IntSlider(
          value=70,
          min=1,
          max=500,
          step=1
    )
    self.par_s = widgets.IntSlider(
          value=3,
          min=1,
          max=15,
          step=1
    )
    self.fcp_s = widgets.FloatSlider(
          value=0.5,
          min=0,
          max=1,
          step=0.05
    )

    self.blurui = widgets.HBox([widgets.VBox([widgets.Label('Exponent Base:'),
                                             widgets.Label('Focus Point:'),
                                             widgets.Label('Number of Partitions:')]),
                                widgets.VBox([self.exp_s, self.fcp_s, self.par_s])])

    self.sliderui = widgets.HBox([self.fileui, self.depthmapui, self.blurui])
    
    out = widgets.interactive_output(self.widget_func, {
        'fileL': self.fl_in,
        'fileR': self.fr_in,
        'num_disp': self.dsp_s,
        'block_size': self.blk_s,
        'exp_base': self.exp_s,
        'focus_point': self.fcp_s,
        'num_partitions': self.par_s
    })
    display(self.sliderui, out)

  def widget_func(self, fileL, fileR, \
                  num_disp, block_size, \
                  exp_base, focus_point, num_partitions):
    self.fileUI(fileL, fileR)
    self.depthMapUI(num_disp, block_size)
    self.blurUI(exp_base, num_partitions, focus_point)
    self.updateUI()

  def fileUI(self, fileL, fileR):
    self.fileL = fileL
    self.fileR = fileR

  def depthMapUI(self, num_disp, block_size):
    self.num_disp = num_disp
    self.block_size = block_size
  
  def blurUI(self, exp_base, num_partitions, focus_point):
    self.exp_base = exp_base
    self.focus_point = focus_point
    self.num_partitions = num_partitions

  def updateUI(self):
    self.lens_blur = simulate_lens_blur(self.fileL, self.fileR, \
                                        self.num_disp, self.block_size, \
                                        self.focus_point, \
                                        self.num_partitions, self.exp_base)
    plt.imshow(cv.cvtColor(self.lens_blur, cv.COLOR_BGR2RGB))

In [17]:
stereoblurUI()

Output()